In [1]:
import pandas as pd
import numpy as np
import ast

In [4]:
credits = pd.read_csv('credits_dataset.csv')
movies = pd.read_csv('movies_dataset.csv')

In [5]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [6]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [7]:
movies = movies.merge(credits, left_on = 'title', right_on = 'title')

In [8]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [9]:
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew', 'original_language', 'production_countries', 'tagline']]

In [10]:
movies.head(1)

,movie_id,title,overview,genres,keywords,cast,crew,original_language,production_countries,tagline
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de...",en,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",Enter the World of Pandora.


In [11]:
def convert(obj):
  L = []
  for i in ast.literal_eval(obj):
    L.append(i['name'])
  return L

In [12]:
movies['genres'] = movies['genres'].apply(convert)
movies['keywords'] = movies['keywords'].apply(convert)
movies['cast'] = movies['cast'].apply(convert)
movies['crew'] = movies['crew'].apply(lambda x: [i['name'] for i in ast.literal_eval(x) if i['job'] == 'Director'])
movies['production_countries'] = movies['production_countries'].apply(convert)

In [13]:
print(movies['genres'])
print(movies['keywords'])
print(movies['cast'])
print(movies['crew'])
print(movies['production_countries'])

0       [Action, Adventure, Fantasy, Science Fiction]
1                        [Adventure, Fantasy, Action]
2                          [Action, Adventure, Crime]
3                    [Action, Crime, Drama, Thriller]
4                [Action, Adventure, Science Fiction]
                            ...                      
4804                        [Action, Crime, Thriller]
4805                                [Comedy, Romance]
4806               [Comedy, Drama, Romance, TV Movie]
4807                                               []
4808                                    [Documentary]
Name: genres, Length: 4809, dtype: object
0       [culture clash, future, space war, space colon...
1       [ocean, drug abuse, exotic island, east india ...
2       [spy, based on novel, secret agent, sequel, mi...
3       [dc comics, crime fighter, terrorist, secret i...
4       [based on novel, mars, medallion, space travel...
                              ...                        
4804    [united 

In [14]:
movies['cast'].head(1)

0    [Sam Worthington, Zoe Saldana, Sigourney Weave...
Name: cast, dtype: object

In [15]:
movies['crew'].head(1)

0    [James Cameron]
Name: crew, dtype: object

In [16]:
movies['tag'] = movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [17]:
movies['tag']

0       [Action, Adventure, Fantasy, Science Fiction, ...
1       [Adventure, Fantasy, Action, ocean, drug abuse...
2       [Action, Adventure, Crime, spy, based on novel...
3       [Action, Crime, Drama, Thriller, dc comics, cr...
4       [Action, Adventure, Science Fiction, based on ...
                              ...                        
4804    [Action, Crime, Thriller, united states–mexico...
4805    [Comedy, Romance, Edward Burns, Kerry Bishé, M...
4806    [Comedy, Drama, Romance, TV Movie, date, love ...
4807    [Daniel Henney, Eliza Coupe, Bill Paxton, Alan...
4808    [Documentary, obsession, camcorder, crush, dre...
Name: tag, Length: 4809, dtype: object

In [18]:
movies = movies[['movie_id', 'title', 'overview', 'original_language', 'production_countries', 'tagline', 'tag']]

In [19]:
movies.head(1)

,movie_id,title,overview,original_language,production_countries,tagline,tag
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",en,"[United States of America, United Kingdom]",Enter the World of Pandora.,"[Action, Adventure, Fantasy, Science Fiction, ..."


In [20]:
movies['tag'] = movies['tag'].apply(lambda x: " ".join(x))

In [21]:
movies['tag'] = movies['tag'].apply(lambda x: x.lower())

In [22]:
movies['tag']

0       action adventure fantasy science fiction cultu...
1       adventure fantasy action ocean drug abuse exot...
2       action adventure crime spy based on novel secr...
3       action crime drama thriller dc comics crime fi...
4       action adventure science fiction based on nove...
                              ...                        
4804    action crime thriller united states–mexico bar...
4805    comedy romance edward burns kerry bishé marsha...
4806    comedy drama romance tv movie date love at fir...
4807    daniel henney eliza coupe bill paxton alan ruc...
4808    documentary obsession camcorder crush dream gi...
Name: tag, Length: 4809, dtype: object

In [23]:
movies.head()

,movie_id,title,overview,original_language,production_countries,tagline,tag
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",en,"[United States of America, United Kingdom]",Enter the World of Pandora.,action adventure fantasy science fiction cultu...
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",en,[United States of America],"At the end of the world, the adventure begins.",adventure fantasy action ocean drug abuse exot...
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,en,"[United Kingdom, United States of America]",A Plan No One Escapes,action adventure crime spy based on novel secr...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,en,[United States of America],The Legend Ends,action crime drama thriller dc comics crime fi...
4,49529,John Carter,"John Carter is a war-weary, former military ca...",en,[United States of America],"Lost in our world, found in another.",action adventure science fiction based on nove...


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies['tag'])

In [25]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [26]:
def get_recommendations(title, cosine_sim=cosine_sim):
  idx = movies[movies['title'] == title].index[0]
  sim_scores = list(enumerate(cosine_sim[idx]))
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
  sim_scores = sim_scores[1:11]
  movie_indices = [i[0] for i in sim_scores]
  return movies['title'].iloc[movie_indices]

In [27]:
print(get_recommendations('Batman Begins'))

3          The Dark Knight Rises
65               The Dark Knight
428               Batman Returns
1360                      Batman
370             Now You See Me 2
591                     Stardust
1384                        TMNT
1297                Superman III
1361                      Batman
4644    Amidst the Devil's Wings
Name: title, dtype: object


In [28]:
import pickle
with open('movies.pkl', 'wb') as file:
  pickle.dump((movies, cosine_sim), file)